In [1]:
import pandas as pd
import tabula as tb
import numpy as np
import shutil
import os

# Putting the pdf files in the folders

In [ ]:
def mover_arquivo(origem, destino):
    try:
        shutil.move(origem, destino)
        print("Arquivo movido com sucesso!")
    except IOError as e:
        print(f"Ocorreu um erro ao mover o arquivo: {e}")
    except Error as e:
        print(f"Ocorreu um erro inesperado: {e}")

lista_meses = ['1-Janeiro', '2-Fevereiro', '3-Março', '4-Abril', '5-Maio', '6-Junho', '7-Julho', '8-Agosto', '9-Setembro', '10-Outubro', '11-Novembro', '12-Dezembro']

In [ ]:
for month in lista_meses:
    # Defina o caminho do arquivo de origem e destino
    origem = f"Demanda - 2019/{month}/{month[0]}.pdf"
    destino = f"Demanda - 2019/PEL/{month[0]}.pdf"

    # Verifique se o arquivo de origem existe
    if os.path.exists(origem):
        mover_arquivo(origem, destino)
    else:
        origem = f"Demanda - 2019/{month}/{month[:2]}.pdf"
        destino = f"Demanda - 2019/PEL/{month[:2]}.pdf"
        mover_arquivo(origem, destino)

In [ ]:
for month in lista_meses:
    # Defina o caminho do arquivo de origem e destino
    origem = f"Demanda - 2020_0/{month[0]}.pdf"
    destino = f"Demanda - 2020_0/PEL/{month[0]}.pdf"

    # Verifique se o arquivo de origem existe
    if os.path.exists(origem):
        mover_arquivo(origem, destino)
    else:
        origem = f"Demanda - 2020_0/{month[:2]}.pdf"
        destino = f"Demanda - 2020_0/PEL/{month[:2]}.pdf"
        mover_arquivo(origem, destino)

# Tests

In [60]:
year = '2017'
month = '10'

In [61]:
df_row = (tb.read_pdf(f"{year}/PEL/{month}.pdf", pages="all"))[0]
df_row = df_row.rename(columns={"DEMANDA (milhares)": "Linha"})

df_row

,Linha,Linha 1 - Azul,Linha 2 - Verde,Linha 3 - Vermelha,Linha 5 - Lilás,Linha 15 - Prata,Rede
0,Total,26.029,13.219,30.557,6.427,260,76.491
1,Média dos dias úteis,1.059,555.000,1.224,264.000,12,3.114
2,Média dos Sábados,569.000,233.000,684.000,149.000,4,1.637
3,Média dos Domingos,291.000,142.000,405.000,57.000,1,895.000
4,Máxima Diária,1.118,569.000,1.288,280.000,12,3.268


In [62]:
df_row.dtypes

Linha                  object
Linha 1 - Azul        float64
Linha 2 - Verde       float64
Linha 3 - Vermelha    float64
Linha 5 - Lilás       float64
Linha 15 - Prata        int64
Rede                  float64
dtype: object

In [63]:
df_row = df_row.T.reset_index()

In [64]:
df_row.head()

,index,0,1,2,3,4
0,Linha,Total,Média dos dias úteis,Média dos Sábados,Média dos Domingos,Máxima Diária
1,Linha 1 - Azul,26.029,1.059,569.0,291.0,1.118
2,Linha 2 - Verde,13.219,555.0,233.0,142.0,569.0
3,Linha 3 - Vermelha,30.557,1.224,684.0,405.0,1.288
4,Linha 5 - Lilás,6.427,264.0,149.0,57.0,280.0


In [65]:
df_row.columns = ['line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']

In [66]:
df_row = df_row[1:6]

In [67]:
df_row

,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,Linha 1 - Azul,26.029,1.059,569.0,291.0,1.118
2,Linha 2 - Verde,13.219,555.0,233.0,142.0,569.0
3,Linha 3 - Vermelha,30.557,1.224,684.0,405.0,1.288
4,Linha 5 - Lilás,6.427,264.0,149.0,57.0,280.0
5,Linha 15 - Prata,260,12,4,1,12


In [68]:
df_row['line'] = df_row['line'].apply(lambda x: int(x.split(' ')[1]))

In [69]:
df = df_row.copy()
df

,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,1,26.029,1.059,569.0,291.0,1.118
2,2,13.219,555.0,233.0,142.0,569.0
3,3,30.557,1.224,684.0,405.0,1.288
4,5,6.427,264.0,149.0,57.0,280.0
5,15,260,12,4,1,12


In [70]:
df['total'] = df['total'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['business_day_mean'] = df['business_day_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['saturday_mean'] = df['saturday_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['sunday_mean'] = df['sunday_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['max'] = df['max'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
df['date'] = pd.to_datetime(year + '-' + month,format='%Y-%m')

df = df[['date', 'line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']]
df

,date,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,2017-10-01,1,26029000,1059000,569000,291000,1118000
2,2017-10-01,2,13219000,555000,233000,142000,569000
3,2017-10-01,3,30557000,1224000,684000,405000,1288000
4,2017-10-01,5,6427000,264000,149000,57000,280000
5,2017-10-01,15,260000,12000,4000,1000,12000


# ETL code

In [2]:
def pel(year, month): 
    df_row = (tb.read_pdf(f"{year}/PEL/{month}.pdf", pages="all"))[0]

    if year == 'Demanda - 2019':
        year = '2019'
    elif year == 'Demanda - 2020_0':
        year = '2020'
        
    df = df_row.T.reset_index()

    df.columns = ['line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']

    df = df[1:6]
    
    try:
        df['line'] = df['line'].apply(lambda x: int(x.replace('\r', ' ').split(' ')[1]) if ' ' in x else x)
    except:
        df['line'] = df['line'].apply(lambda x: int(x.replace('-', ' ').replace('\r', ' ').split(' ')[1]) if ' ' in x else x)

    
    df['total'] = df['total'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['business_day_mean'] = df['business_day_mean'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['saturday_mean'] = df['saturday_mean'].apply(lambda x: (np.NaN if str(x) == '-' else int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['sunday_mean'] = df['sunday_mean'].apply(lambda x: (np.NaN if str(x) == '-' else int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    df['max'] = df['max'].apply(lambda x: (int(x) if str(x)[-2:] == '.0' else  int(str(x).replace(',', '').replace('.', '')) if isinstance(x, float) else x) * 1000)
    
    df['date'] = pd.to_datetime(year + '-' + month,format='%Y-%m')

    df = df[['date', 'line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']]

    return df

In [97]:
years = ['2017', '2018', 'Demanda - 2019', 'Demanda - 2020_0']

year_df = []
for year in years:
    if year == '2017':
        for month in range(10,13):
            df_processed = pel(year, str(month))
            year_df.append(df_processed)
    else:
        for month in range(1,13):
            df_processed = pel(year, str(month))
            year_df.append(df_processed)
row_df = pd.concat(year_df)


In [5]:
#dataset = pd.concat(year_df)
row_df.head(20)

,date,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,2017-10-01,1,26029000,1059000,569000.0,291000,1118000
2,2017-10-01,2,13219000,555000,233000.0,142000,569000
3,2017-10-01,3,30557000,1224000,684000.0,405000,1288000
4,2017-10-01,5,6427000,264000,149000.0,57000,280000
5,2017-10-01,15,260000,12000,4000.0,1000,12000
1,2017-11-01,1,24834000,1077000,571000.0,319000,1116000
2,2017-11-01,2,12437000,559000,229000.0,146000,574000
3,2017-11-01,3,29305000,124000,709000.0,444000,1274000
4,2017-11-01,5,6348000,274000,155000.0,77000,285000
5,2017-11-01,15,269000,13000,4000.0,1000,14000


In [18]:
filter1 = row_df['line']!='Rede'

df_filtred = row_df.loc[filter1]
df_filtred = df_filtred.loc[df_filtred['line']!= 5]


In [19]:
df_filtred

,date,line,total,business_day_mean,saturday_mean,sunday_mean,max
1,2017-10-01,1,26029000,1059000,569000.0,291000,1118000
2,2017-10-01,2,13219000,555000,233000.0,142000,569000
3,2017-10-01,3,30557000,1224000,684000.0,405000,1288000
5,2017-10-01,15,260000,12000,4000.0,1000,12000
1,2017-11-01,1,24834000,1077000,571000.0,319000,1116000
...,...,...,...,...,...,...,...
4,2020-11-01,15,1002000,40000,27000.0,15000,43000
1,2020-12-01,1,12855000,514000,328000.0,173000,557000
2,2020-12-01,2,6482000,267000,139000.0,81000,288000
3,2020-12-01,3,16328000,647000,423000.0,235000,719000


In [21]:
df_filtred.to_csv('test.csv', index=False)

In [22]:
pd.set_option('display.max_rows', None)

# Merging the datasets

In [79]:
df_new = pd.read_csv('publiclines_pel_21-23.csv')
df_new = df_new.sort_values('date')

In [80]:
dfs_list = [df_filtred, df_new]
complete_df = pd.concat(dfs_list)

In [81]:
complete_df = complete_df.reset_index()[['date', 'line', 'total', 'business_day_mean', 'saturday_mean', 'sunday_mean', 'max']]

In [95]:
complete_df.sample(5)

,date,line,total,business_day_mean,saturday_mean,sunday_mean,max
3,2017-10-01,15,260000,12000,4000.0,1000,12000
17,2018-02-01,2,11847000,544000,282000.0,180000,583000
227,2022-06-01,3,20959000,823000,514000.0,322000,858000
73,2019-04-01,2,15516000,648000,267000.0,170000,659000
143,2020-09-01,15,882000,35000,23000.0,12000,37000


In [83]:
na_numbers = [43, 51, 119, 123, 127] #the number of each invalid row

for n in na_numbers:
    complete_df.loc[n, 'total'] = np.NaN
    complete_df.loc[n, 'business_day_mean'] = np.NaN
    complete_df.loc[n, 'saturday_mean'] = np.NaN
    complete_df.loc[n, 'sunday_mean'] = np.NaN
    complete_df.loc[n, 'max'] = np.NaN



In [90]:
complete_df['date'] = pd.to_datetime(complete_df['date']).dt.strftime('%Y-%m-%d')

In [91]:
complete_df.to_csv('publiclines_pel_17-23.csv', index=False)